In [1]:
import pandas as pd
import os
import glob
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
pd.options.mode.chained_assignment = None
from scipy import stats
from tqdm import tqdm
sns.set_theme()
sns.set(font_scale=1)

EPSILON = 0.001
PHYS_PROPERTY = {'equate_1':'AD-controlled', 'equate_2': 'TS-controlled', 'equate_3': 'CH-controlled'}
PHYS_PROPERTY_TO_NUM = {'Average Diameter':1, 'Total Surface Area':2, 'Convex Hull': 3}
NUM_TO_PHYS_PROPERTY = {1: 'AD-controlled', 2: 'TS-controlled', 3: 'CH-controlled'}
EXPERIMENTS = { 'size-count', 'count-size', 'colors-count'}
CONGRUENT_COLUMNS = ['Ratio 50 Congruent Validation Accuracy',
                     'Ratio 56 Congruent Validation Accuracy',
                     'Ratio 63 Congruent Validation Accuracy',
                     'Ratio 71 Congruent Validation Accuracy',
                     'Ratio 75 Congruent Validation Accuracy',
                     'Ratio 86 Congruent Validation Accuracy']
INCONGRUENT_COLUMNS = ['Ratio 50 Incongruent Validation Accuracy',
                     'Ratio 56 Incongruent Validation Accuracy',
                     'Ratio 63 Incongruent Validation Accuracy',
                     'Ratio 71 Incongruent Validation Accuracy',
                     'Ratio 75 Incongruent Validation Accuracy',
                     'Ratio 86 Incongruent Validation Accuracy']
TO_REMOVE = 9999999.9
STD_FILTER = 2.5
CONGRUENT_VALUE = 1
INCONGRUENT_VALUE = 0
CONGRUENCY = {CONGRUENT_VALUE: "Congruent", INCONGRUENT_VALUE: "Incongruent"}
RATIOS = ['50','56','63','71','75','86']

In [5]:
def split_train_test_results(files_names):
    tests = []
    for file in files_names:
        if "Tested_on" in file:
            tests.append(file)
        else:
            train = file
    return tests, train

Train and test result files as a dict

Aggregate all files into dict

In [9]:
PATH = "/Users/gali.k/phd/phd_2021/post_tests/result_proffeciancy_0.65_counting_stop_0.80__batch_512_2_epochs"

result_dict = {}
for equate in PHYS_PROPERTY.keys():
    equate_dict = {}

    for experiment in EXPERIMENTS:
        curr_results_path = PATH + os.sep + equate + os.sep + experiment
        result_file_names = glob.glob(curr_results_path + os.sep + f"Results_*.csv")
        if len(result_file_names) > 0:
            testing_file_name1  = result_file_names[0]
            testing_file_1_df =  pd.read_csv(testing_file_name1)
            testing_file_1_df['Tested_On_Equate'] = testing_file_name1[testing_file_name1.find('Tested_on'): testing_file_name1.find('AvgAccuracy')-1].replace('Tested_on_Equate', 'equate')
            equate_dict.update({experiment: { "test_1": testing_file_1_df,}})
            result_dict.update({equate: equate_dict})

Preprocess the dataframe

Create ANOVA CSV

In [10]:
def prepare_ratios_df(curr_df):
    ratios_df = pd.DataFrame()
    for ratio in RATIOS:
        ratio_cong_df = pd.DataFrame()
        ratio_cong_df['UNIQUE_SUBJECT_UID'] = curr_df['UNIQUE_SUBJECT_UID']
        ratio_cong_df['Task'] = curr_df['Task']
        ratio_cong_df['Generations'] = curr_df['Generations']
        ratio_cong_df['Train'] = curr_df['Equate'].apply(lambda val: NUM_TO_PHYS_PROPERTY[val])
        ratio_cong_df['Test'] = curr_df['Tested_On_Equate'].apply(lambda val: PHYS_PROPERTY[val])
        ratio_cong_df['Congruity'] = 'Congruent'
        ratio_cong_df['Ratio'] = ratio
        ratio_cong_df['Training Accuracy'] = curr_df[f'Ratio {ratio} Congruent Training Accuracy']
        ratio_cong_df['Validation Accuracy'] = curr_df[f'Ratio {ratio} Congruent Validation Accuracy']
        ratio_cong_df['Training Loss'] = curr_df[f'Ratio {ratio} Congruent Validation Loss']
        ratio_cong_df['Validation Loss'] = curr_df[f'Ratio {ratio} Congruent Validation Loss']
        ratio_cong_df['Layers'] = curr_df['Layers']
        ratio_cong_df['Neurons'] = curr_df['Nuerons']
        ratio_cong_df['Activation'] = curr_df['Activation']
        ratio_cong_df['Optimizer'] = curr_df['Optimizer']

        ratio_incong_df = pd.DataFrame()
        ratio_incong_df['UNIQUE_SUBJECT_UID'] = curr_df['UNIQUE_SUBJECT_UID']
        ratio_incong_df['Task'] = curr_df['Task']
        ratio_incong_df['Generations'] = curr_df['Generations']
        ratio_incong_df['Train'] = curr_df['Equate'].apply(lambda val: NUM_TO_PHYS_PROPERTY[val])
        ratio_incong_df['Test'] = curr_df['Tested_On_Equate'].apply(lambda val: PHYS_PROPERTY[val])
        ratio_incong_df['Congruity'] = 'Incongruent'
        ratio_incong_df['Ratio'] = ratio
        ratio_incong_df['Training Accuracy'] = curr_df[f'Ratio {ratio} Incongruent Training Accuracy']
        ratio_incong_df['Validation Accuracy'] = curr_df[f'Ratio {ratio} Incongruent Validation Accuracy']
        ratio_incong_df['Training Loss'] = curr_df[f'Ratio {ratio} Incongruent Validation Loss']
        ratio_incong_df['Validation Loss'] = curr_df[f'Ratio {ratio} Incongruent Validation Loss']
        ratio_incong_df['Layers'] = curr_df['Layers']
        ratio_incong_df['Neurons'] = curr_df['Nuerons']
        ratio_incong_df['Activation'] = curr_df['Activation']
        ratio_incong_df['Optimizer'] = curr_df['Optimizer']

        ratio_df = pd.concat([ratio_cong_df, ratio_incong_df])
        ratios_df = pd.concat([ratios_df, ratio_df])
    return ratios_df.reset_index(drop=True)

In [11]:
result_dict.keys()
anova_df = pd.DataFrame()
mixed_anova_df = pd.DataFrame()
for phys_prop in tqdm(result_dict.keys()):
    #print(f"Working on {phys_prop}")
    exp_result = result_dict[phys_prop]
    for task in exp_result.keys():
        #print(f"Task: {task}")

        test1_df = exp_result[task]['test_1']
        test1_df['UNIQUE_SUBJECT_UID'] = test1_df['Subject_UID'].apply(lambda x: task + "_" + phys_prop + "_" + str(x))
        test_1_ratios_df = prepare_ratios_df(test1_df)
        mixed_anova_df = pd.concat([mixed_anova_df, test_1_ratios_df])

mixed_anova_df.to_csv('mixed_anova_df.csv')

100%|██████████| 3/3 [00:00<00:00,  4.44it/s]


In [12]:
len(mixed_anova_df)

2160

Filter nulls

In [13]:
non_na_mixed_anova = mixed_anova_df.dropna()
len(non_na_mixed_anova)

2136

In [14]:
non_na_mixed_anova.describe()

,Generations,Training Accuracy,Validation Accuracy,Training Loss,Validation Loss,Layers
count,2136.000000,2136.000000,2136.000000,2136.000000,2136.000000,2136.000000
mean,11.747191,0.515169,0.515169,1.800996,1.800996,3.292135
std,8.512272,0.349830,0.349830,7.799110,7.799110,1.138985
min,3.000000,0.000000,0.000000,0.000300,0.000300,2.000000
25%,6.000000,0.200000,0.200000,0.575025,0.575025,2.000000
50%,6.000000,0.500000,0.500000,0.934700,0.934700,3.000000
75%,19.000000,0.850000,0.850000,1.286625,1.286625,4.000000
max,30.000000,1.000000,1.000000,142.571000,142.571000,5.000000


In [15]:
non_na_mixed_anova.columns

Index(['UNIQUE_SUBJECT_UID', 'Task', 'Generations', 'Train', 'Test',
       'Congruity', 'Ratio', 'Training Accuracy', 'Validation Accuracy',
       'Training Loss', 'Validation Loss', 'Layers', 'Neurons', 'Activation',
       'Optimizer'],
      dtype='object')

Filter extreme values

In [16]:
anova_df_for_R = non_na_mixed_anova[['UNIQUE_SUBJECT_UID', 'Task','Train','Test','Generations','Congruity','Ratio','Validation Accuracy']]
anova_df_for_R.head()

,UNIQUE_SUBJECT_UID,Task,Train,Test,Generations,Congruity,Ratio,Validation Accuracy
0,size-count_equate_1_115,size,AD-controlled,AD-controlled,6,Congruent,50,0.0
1,size-count_equate_1_112,size,AD-controlled,AD-controlled,6,Congruent,50,0.0
2,size-count_equate_1_107,size,AD-controlled,AD-controlled,6,Congruent,50,0.0
3,size-count_equate_1_110,size,AD-controlled,AD-controlled,6,Congruent,50,0.0
4,size-count_equate_1_101,size,AD-controlled,AD-controlled,6,Congruent,50,0.1


In [17]:
anova_df_for_R.to_csv("simulation_7_anova_df_for_R_26_01_23.csv")

Total Generations data

In [18]:
anova_df_for_R_total_generations = anova_df_for_R[['UNIQUE_SUBJECT_UID', 'Task','Train', 'Generations']].groupby(['UNIQUE_SUBJECT_UID', 'Task','Train']).mean()

In [19]:
anova_df_for_R_total_generations.to_csv("simulation_7_anova_df_for_R_23_01_23_generations.csv")

Separating to physical and numerical

************* Until here for the R anova analysis! **************

In [31]:
overall_means_1 = anova_df_for_R[['Task','Congruity', 'Validation Accuracy']].groupby(['Task','Congruity']).agg(['mean','std'])

In [32]:
overall_means_1.head(10)

Validation Accuracy          
                                  mean       std
Task   Congruity                                
colors Congruent              0.488842  0.113886
       Incongruent            0.509322  0.106771
count  Congruent              0.926250  0.178077
       Incongruent            0.110833  0.154575
size   Congruent              0.733051  0.335793
       Incongruent            0.322599  0.349156

In [33]:
overall_means_2 = anova_df_for_R[['Task','Congruity', 'Train', 'Validation Accuracy']].groupby(['Task','Train','Congruity']).agg(['mean','std'])

In [34]:
overall_means_2.head(30)

Validation Accuracy          
                                                mean       std
Task   Train         Congruity                                
colors AD-controlled Congruent              0.516667  0.107166
                     Incongruent            0.533333  0.080266
       CH-controlled Congruent              0.464912  0.126023
                     Incongruent            0.537719  0.091342
       TS-controlled Congruent              0.483750  0.102728
                     Incongruent            0.458333  0.124403
count  AD-controlled Congruent              0.949583  0.166337
                     Incongruent            0.112917  0.169960
       CH-controlled Congruent              0.922083  0.206389
                     Incongruent            0.075000  0.129479
       TS-controlled Congruent              0.907083  0.156417
                     Incongruent            0.144583  0.154865
size   AD-controlled Congruent              0.829167  0.359480
                     Incongruent            0.220833  0.352755
       CH-controlled Congruent              0.798246  0.403082
                     Incongruent            0.199123  0.398503
       TS-controlled Congruent              0.575000  0.111333
                     Incongruent            0.541667  0.113821

In [35]:
overall_means_3 = anova_df_for_R[['Task','Congruity', 'Train', 'Validation Accuracy']].groupby(['Task']).agg(['mean','std'])
overall_means_3.head(30)

/var/folders/bw/951w4nh12991sq82d74c_gv9gj4m7h/T/ipykernel_34927/3262724012.py:1: FutureWarning: ['Congruity', 'Train'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  overall_means_3 = anova_df_for_R[['Task','Congruity', 'Train', 'Validation Accuracy']].groupby(['Task']).agg(['mean','std'])


Validation Accuracy          
                      mean       std
Task                                
colors            0.499082  0.110783
count             0.518542  0.440705
size              0.527825  0.399180